In [40]:
import numpy as np
import pandas as pd

In [43]:
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from urb3d.segmentation.dataset import PointCloudSegmentationDataset, PointSampler, ChunkedPointCloudDataset
from urb3d.segmentation.segmentor import PointNetSegmentor

from sklearn.metrics import precision_recall_fscore_support

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [44]:
weighted_model = '../models/pointnet_weighted/best_results/epoch=31-val_loss=1.99-train_loss=1.35.ckpt'
unweighted_model = '../models/new_deal/best_results/epoch=89-val_loss=1.81-train_loss=1.32.ckpt'
# block_6_model = '../models/longrun_pointnet_weighted_ins/best_results/epoch=1-val_loss=2.42-train_loss=2.51.ckpt'
block_7_model = '../models/pointnet_weighted_isns/best_results/epoch=113-val_loss=0.79-train_loss=1.18.ckpt'
block_6_model = ('../models/pointnet_weighted_isns_6/best_results/epoch=45-val_loss=1.22-train_loss=1.41.ckpt')
block_6_7_model = ('../models/pointnet_weighted_isns_6_7/best_results/epoch=64-val_loss=2.01-train_loss=1.34.ckpt')

In [100]:
model = PointNetSegmentor.load_from_checkpoint(weighted_model, strict=False)
dataset = ChunkedPointCloudDataset('../data/birmingham_blocks/val')
data_loader = DataLoader(dataset)
trainer = pl.Trainer()
preds = trainer.predict(model, data_loader)
# preds = preds[0][0].cpu().numpy()
preds = preds
#labels = data_loader.__iter__().__next__()[1][0].cpu().numpy()

C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['criterion.weight']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
labels = np.array([])
predictions = np.array([])

for idx, chunk in enumerate(data_loader):
    chunk_labels = chunk[1][0].cpu().numpy()
    labels = np.concatenate([labels, chunk_labels])
    chunk_preds = preds[idx][0].cpu().numpy()
    predictions = np.concatenate([predictions, chunk_preds])


In [96]:
metrics = precision_recall_fscore_support(labels, predictions)
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()
print(precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=0))

C:\Users\kasia\PycharmProjects\urban-3d-reconstruction\venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


0.29		0.3		0.29		4820801		
0.05		0.2		0.08		829892		
0.36		0.2		0.26		6047457		
0.03		0.09		0.05		542358		
0.05		0.08		0.06		900471		
0.15		0.03		0.06		2534264		
0.02		0.01		0.02		377259		
0.04		0.06		0.05		659178		
0.0		0.0		0.0		0		
(0.24560487416272858, 0.18315746830958946, 0.197450849455238, None)


In [99]:
for class_id in range(len(metrics[0])):
    for metric in range(4):
        print(round(metrics[metric][class_id],2), end='\t\t')
    print()

print(precision_recall_fscore_support(labels, preds, average='weighted'))

0.29		0.3		0.29		4820801		
0.05		0.2		0.08		829892		
0.36		0.2		0.26		6047457		
0.03		0.09		0.05		542358		
0.05		0.08		0.06		900471		
0.15		0.03		0.06		2534264		
0.02		0.01		0.02		377259		
0.04		0.06		0.05		659178		
0.0		0.0		0.0		0		


ValueError: Found input variables with inconsistent numbers of samples: [16711680, 255]

In [ ]:
2

In [19]:
labels.shape

torch.Size([1, 238950])

In [11]:
preds[0][0].shape

torch.Size([238950])

In [9]:
for pred in preds:
    print(pred.shape)

torch.Size([1, 238950])


In [9]:
len(data_loader.dataset.pt.points)

238950

In [7]:
data_loader.dataset.point_sampler.subsample_size

In [15]:
print(data_loader.dataset.subsample_size)

238950


In [6]:
data_loader.__iter__().__next__()[0].shape

torch.Size([1, 3])

In [13]:
preds[0].shape

torch.Size([1, 1024])